In [ ]:
from gpt_4o_mini import DataPreparation, ClassificationModule, Trainer, GPT4Model, Evaluator
from utilities import Config
import os
import dotenv

In [ ]:
dotenv.load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
proxy_url = os.getenv("PROXY_URL")

config = Config(train_size=20, test_size=20)
data_prep = DataPreparation(config)
data_prep.load_data("../data/open_domain_data.csv", "../data/specific_domain_data.csv")

gpt_model = GPT4Model(api_key=api_key, proxy_url=proxy_url, )
evaluator = Evaluator(gpt_model)

trainer = Trainer(ClassificationModule, data_prep.train_data, evaluator)
compiled_model = trainer.optimize_model()
print("Model optimized and compiled.")

predictions = []
true_labels = [example.label for example in data_prep.test_data]
total_tokens = 0

for example in data_prep.test_data:
    prompt = example.prompt
    prediction = compiled_model(prompt)
    predictions.append(prediction.label)

    total_tokens += evaluator.count_tokens(prompt)
    total_tokens += evaluator.count_tokens(prediction.label)

accuracy = evaluator.evaluate_model(predictions, true_labels)
price = evaluator.calculate_price(total_tokens)

print(f"Evaluation results: Accuracy {accuracy * 100:.2f}%")
print(f"Total price: ${price:.4f} USD")